In [1]:
import pandas as pd
from transformers import pipeline
from modules.tweet_data import read_raw_data
from modules.spacy import spacy_twitter_model
import datetime as dt

In [2]:
nlp = spacy_twitter_model()
sentiment_model = pipeline('sentiment-analysis')

In [3]:
tweet_df = read_raw_data()

Reading data


In [4]:
es = pd.read_csv('data/ES.csv')
es = es[es.Time == '15:00'][['Date','Time','Close']]
es.Date = pd.to_datetime(es.Date +' ' +'15:45')
es.drop('Time',axis=1,inplace=True)
es.set_index('Date', inplace=True)
date_range = es['2018-6-1':'2020-5-31'].index

In [81]:
def twitter_tokenizer(d,
                      model=nlp,
                      urls=True,
                      stop_words=False,
                      lowercase=True,
                      alpha_only=True,
                      hashtags=True,
                      lemma=False):
    """Full tokenizer with flags for processing steps
    urls: If True, remove URLs
    stop_words: If True, removes stop words
    lowercase: If True, lowercases all tokens
    alpha_only: If True, removes all non-alpha characters
    lemma: If True, lemmatizes words

    The tokenizer also removes all URLS
    """
    parsed = model(d)
    # token collector
    tokens = []
    for t in parsed:
        # remove URLs
        if t.like_url or t._.is_piclink & urls:
            continue
        # only include stop words if stop words==True
        if t.is_stop & stop_words:
            continue
        # only include non-alpha is alpha_only==False
        if not t.is_alpha & alpha_only:
            if hashtags:
                continue
            else:
                if not t._.is_hashtag:
                    continue
        if lemma:
            t = t.lemma_
        else:
            t = t.text
        if lowercase:
            t = t.lower()
        tokens.append(t)
    return tokens

In [6]:
sample = tweet_df[date_range[0]:(date_range[1]-dt.timedelta(seconds=1))]

In [37]:
text = ' '.join(sample.tweet)

In [17]:
tweet_df

,tweet_id,handle,tweet
timestamp,,,
2006-09-22 09:02:46,33743,guan,"Hello, World!"
2006-09-22 09:21:33,33745,guan,Working on a cool new project at Steffen's house.
2007-01-17 21:43:23,3175083,MarketWatch,Ex-Cendant chairman gets 12-year-plus prison s...
2007-01-17 21:43:25,3175143,MarketWatch,Ex-Cendant chairman gets 12-year prison senten...
2007-01-17 21:44:50,3175323,MarketWatch,Home builders' sentiment rises to highest leve...
...,...,...,...
2020-07-10 20:32:55,1281687871406186497,RANsquawk,Cheers good sir!!! you too mate :)
2020-07-10 20:37:10,1281688941389905921,lindayueh,"Prime Minister, who posed in a face covering f..."
2020-07-10 20:52:55,1281692904071876611,bopinion,"Today, roughly 20% of all U.S.-listed stocks a..."


In [19]:
sample.sample(1000)

,tweet_id,handle,tweet
timestamp,,,
2018-06-02 03:00:19,1002746707493425152,AJEnglish,Personal stories from around the world about d...
2018-06-03 11:04:28,1003230933183590401,majilato,Suppose you came to terms with going bald. .....
2018-06-03 16:36:45,1003314556498739201,ErikFossing,For sure. These are charges by the Commission...
2018-06-03 17:09:42,1003322850051346432,Redistrict,Another big problem for Dems in #CA39 is that ...
2018-06-02 03:00:01,1002746629726916608,FreeBeacon,George Soros Pulls Out of San Diego’s District...
...,...,...,...
2018-06-04 02:51:14,1003469195424694272,prchovanec,"Jun 3, 1918 - New York Times: US fighter pilot..."
2018-06-03 12:29:38,1003252365598904321,adamscrabble,.@johnrobb not sure if youre following the li...
2018-06-04 14:01:49,1003637954986864640,rcallimachi,"An unusual partnership: @_JesseMorton, once a ..."


In [9]:
import time

In [82]:
t = time.time()
ww = [' '.join(twitter_tokenizer(tweet)) for tweet in sample.tweet]
print(time.time() - t)

129.0026581287384


In [87]:
t = time.time()
sample_size = len(ww)
batch_size = 4
samp = ww
samp = [list(samp[i:i+batch_size]) for i in range(0,sample_size,batch_size)]
t = time.time()
for s in samp:
    A.append(sentiment_model(s))
print(time.time() - t)

359.92059326171875


In [51]:
A= []
samp = sample.tweet.sample(1000)
samp = [list(samp[i:i+10]) for i in range(0,1000,10)]

In [45]:
samp[0]

['Why a bet against the dollar makes sense as trade-war fears fester https://on.mktw.net/2J9ufnK\xa0',
 'michael burryesque',
 'France, Ifop poll:  President Macron Approval Rating  Approve: 41% (-3) Disapprove: 57% (+2)  Field work: 17/05/18 – 26/05/18 Sample size: 1,933',
 'Shoutout to @larissaluu @andy_mills @schwendydorr and @Pasthaaa !',
 '#IAEA Urges Cooperation from #Iran over Nuclear Inspections  http://aawsat.com/node/1289681\xa0',
 "I've thought about it for a long time and I objectively cannot come up with a reason why someone would need to spend more than a few dollars on a set of pens.",
 "Now that we know that the White House repeatedly lied about Trump's dictation of Don Trump Jr's false statements about the Trump Tower meeting ... isn't it time to review WH denials that it blocked the TimeWarner/ATT merger for malicious political reasons http://www.chicagotribune.com/business/ct-biz-doj-lawsit-time-warner-att-20171121-story.html\xa0…",
 'Israel’s finance ministry is pro

In [65]:
sample.tweet[sample.tweet.str.len()>10]

timestamp
2018-06-01 15:45:03    Wife of ex-Malaysian PM summoned in probe over...
2018-06-01 15:45:08    It's the collective action that matters and in...
2018-06-01 15:45:12    But you know, at 20, my heroes were HL Mencken...
2018-06-01 15:45:19    It’s our 20th birthday today! Want to know mor...
2018-06-01 15:45:21                           *A funeral. I cannot type.
                                             ...                        
2018-06-04 15:44:40    The unfilled vacancy for benign hegemon of glo...
2018-06-04 15:44:44    $AMD so much call buying last month or so smel...
2018-06-04 15:44:45    Bezoek OCMW van Etterbeek, ontmoeting met maat...
2018-06-04 15:44:49    Also - the Government needs to pass not just 1...
2018-06-04 15:44:55    Sure.  But what does that have to do with his ...
Name: tweet, Length: 22850, dtype: object

In [60]:
t = time.time()
sample_size = 2048
batch_size = 4
samp = sample.tweet.sample(sample_size)
samp = [list(samp[i:i+batch_size]) for i in range(0,sample_size,batch_size)]
t = time.time()
for s in samp:
    A.append(sentiment_model(s))
print(time.time() - t)

63.15687084197998


In [53]:
t = time.time()
sentiment_model(list(sample.tweet.sample(sample_size)))
print(time.time() - t)

92.40196514129639


In [28]:
score = 0
score = sum([a['score'] if a['label'] == 'POSITIVE' else -a['score'] for a in A])

    

In [29]:
score/1000

-0.45797855550050737